# Module 5: Transfer learning - Practice

In this session, you will create a **deep convolutional neural network** (DCNN)
to distinguish **dogs vs cats** images.
You will be able to train a very accurate model based on a dataset containing 20k+ images using
a deep learning technique called transfer learning, which is typically done because of:

1. Difficulties in data acuisition or limited amount of data
2. Prohibitively long training duration of a highly complex model

The Transfer Learning technique is to reuse an existing DCNN as **feature extraction**
along with its trained weights for solving a problem in a slightly different domain.
Deep convolutional networks exploit the hierarchical distributed feature representations where,
lower levels of layers contain more generic features (e.g. edges or blobs) that should be useful to many tasks,
while higher levels of layers extracts information relevant to some particular application of the network.

Read more in this very popular article: [Building powerful image classification models using very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html).

In this session we will keep using **Keras** to create a deep convolutional network from high level.

Keras documentation click [here](https://keras.io/layers/core/).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools, functools
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K

import tf_threads
tfconfig = tf_threads.limit(tf, 2)
session = tf.Session(config=tfconfig)
K.set_session(session)

import h5py
from skimage.io import imread, imshow
from skimage.transform import resize
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Reshape, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16

# Load dataset

Here's directory structure for this dataset:
~~~
DogsCats/
    train/
        cat.001.jpg
        cat.002.jpg
        dog.001.jpg
        dog.002.jpg
~~~

Run this cell to show a cat image!

In [ ]:
DATASET = lambda fname = '': os.path.join('/dsa/data/all_datasets/transfer_learning/DogsCats', fname)
assert os.path.exists(DATASET())

plt.figure()

# Study This Statement:
print("Showing the train/cat.100.jpg, full path : {}".format(
            DATASET('train/cat.100.jpg')
        )
     )

imshow(imread(DATASET('train/cat.100.jpg')))

In [ ]:
help(imshow)

### Show another image of a dog, say '`dog.200.jpg`' from the training folder.

In [ ]:
# Add code below this comment  (Question #P5101)
# ----------------------------------









# Create a DCNN model based on VGG16

In the following cell, you will create a computational graph.
This time we will transfer [VGG Networks](http://www.robots.ox.ac.uk/~vgg/research/very_deep/),
[or academic paper: VGG16](https://arxiv.org/abs/1409.1556) network along with its
weights instead of creating the neural network from scratch.
But we will detach dense layers from VGG16 network and add our own.
During training, all the weights within VGG16 will stay locked, so we'll be using 
whatever features VGG16 gives us.
Only weights in the dense layers will be trained, and this part serves as a classifier.

**Tip**: _If you want some hints and review, please go back to the [MNIST lab](../labs/MNIST.ipynb)._

Steps:

1. Specify input shape (150, 150, 3)
2. Lock weights in VGG16 by setting each
```
layer.trainable = False
```
3. Complete dense layers
  * the 1st one will have **256** units of neuron with **relu** activation.
  * the 2nd one will have **1** unit of neuron with **sigmoid** activation.

In [ ]:
# Complete code below this comment  (Question #P5102)
# ----------------------------------
images = Input(shape = <placeholder>)

vgg16 = VGG16(weights='imagenet', include_top=False)

# Lock the VGG16 Layers
for layer in vgg16.layers:
    <placeholder>   # Look above

classifier = [
    Flatten(input_shape = vgg16.output_shape[1:]),
    
    # Size, and Neuron Type
    Dense(<placeholder>, activation=<placeholder>, name = 'dense_1'),
    Dropout(0.5),

    # Size, and Neuron Type
    Dense(<placeholder>, activation=<placeholder>, name = 'dense_2'),
]

y_pred = functools.reduce(lambda f1, f2: f2(f1), [images, vgg16]+classifier)

model = Model(inputs = [images], outputs = [y_pred])

#### Compile this DCNN model.

* **'binary_crossentropy'** as loss function
* SGD optimizer with
  * **learning rate (lr)  = $1 \times 10^{-4}$ ** and
  * **momentum = 0.9**.
* For `metrics` use only :
```
['accuracy']
```

In [ ]:
# Complete code below this comment  (Question #P5103)
# ----------------------------------
model.compile(loss=<placeholder>,
    optimizer=SGD(lr=<placeholder>, momentum=<placeholder>),
    metrics=<placeholder>)

Print a model summary. 
Notice that number of trainable params should be different from total params, 
which indicates that you have locked VGG16 successfully.

In [ ]:
# Add code below this comment  (Question #P5104)
# ----------------------------------






# Training a DCNN model using transfer learning

Now we are going to create generators that generate batches for us.

* set batch size = **20**
* set image resolution to **(150,150)** (this is what target_size is referring to)
* set class mode as **'binary'** because there are only two classes: dog and cat

In [ ]:
# Complete code below this comment  (Question #P5105)
# ----------------------------------
BATCH_SIZE = <placeholder>

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1. / 255)

In [ ]:
# Complete code below this comment  (Question #P5106)
# ----------------------------------
train_generator = train_datagen.flow_from_directory(
    DATASET('TransferLearning/train'),
    target_size=<placeholder>,
    batch_size=BATCH_SIZE,
    class_mode=<placeholder>)

validation_generator = test_datagen.flow_from_directory(
    DATASET('TransferLearning/validation'),
    target_size=<placeholder>,
    batch_size=BATCH_SIZE,
    class_mode=<placeholder>)

#### Fit this DCNN.

<span style="background:yellow">**NOTE:** This will take a **couple hours** for 1 epochs, depending on system load.</span>

You should let it run a bit, then move on with loading the weights we have trained for you.

You can interupt with `Kernel > Interrupt`

In [ ]:
try:
    model.fit_generator(
        train_generator, steps_per_epoch = 12500 // BATCH_SIZE,
        validation_data=validation_generator, validation_steps=800 // BATCH_SIZE,
        epochs=1) # Normally a few more epochs will be expected.
except KeyboardInterrupt:
    """Select from top menu Kernel->Interrupt to quit training and
    leave model parameters (or weights) as they are."""

If you let this run through a few epochs, you will see lines such as this:

```
Epoch 1/5
625/625 [==============================] - 6335s - loss: 0.5368 - acc: 0.7282 - val_loss: 0.3688 - val_acc: 0.8675
Epoch 2/5
625/625 [==============================] - 6429s - loss: 0.4119 - acc: 0.8121 - val_loss: 0.2991 - val_acc: 0.8875
```

### Saving the learned model

If you let
Here how to save this model. 
This requires that `h5py` has been installed. 

In [ ]:
model.save_weights('./weights_dogs_cats.h5')
os.path.exists('./weights_dogs_cats.h5')

# Evaluation

Now that we've gone through the creation of a model, 
we could define a class that allows us to reuse this model more easily.

Repeat exactly what we did before to **complete this DogsVsCats classs**.

In [ ]:
# Complete code below this comment  (Question #P5107)
# ----------------------------------
class DogsVsCats(Model):
    def __init__(self):
        self.images = Input(<placeholder>)
        self.vgg16 = VGG16(<placeholder>)
        
        classifier = <placeholder>

        self.prediction = functools.reduce(lambda f1, f2: f2(f1), [self.images, self.vgg16]+classifier)
        
        super(DogsVsCats, self).__init__(
            inputs = [self.images],
            outputs = [self.prediction]
        )
        
        self.compile(<placeholder>)
        
    def freeze_vgg16(trainable = False):
        for layer in self.vgg16.layers:
            layer.trainable = trainable

Create a new model and load in these weights we just saved.

load_weights() will only need **a file path** as an argument.

In [ ]:
# Complete code below this comment  (Question #P5108)
# ----------------------------------
new_model = DogsVsCats()
new_model.load_weights(<placeholder>)

Evaluate this model. Use **validation_generator** to provide data.

In [ ]:
# Complete code below this comment  (Question #P5109)
# ----------------------------------
loss, accuracy = new_model.evaluate_generator(<placeholder>, steps = 800 // BATCH_SIZE)
print('loss:', loss, 'accuracy:', accuracy)

Here's how you can make a prediction for one image using this model.

In [ ]:
im_test = imread(DATASET('test1/5.jpg'))
imshow(im_test)
im_test = resize(im_test, (150, 150), mode = 'reflect')
y_pred = new_model.predict(np.expand_dims(im_test, 0)).squeeze()
print(['Cat', 'Dog'][y_pred>=0.5])

## Create another model with DogsVsCats()
... and load in some weights that we have trained for you over 50 epochs at  
"`/dsa/data/all_datasets/transfer_learning/DogsCats/weights_dogs_cats.h5"`

Name the new model <span style="background: yellow;">"pretrained_model"</span>.

In [ ]:
assert os.path.exists('/dsa/data/all_datasets/transfer_learning/DogsCats/weights_dogs_cats.h5')

# Add code below this comment  (Question #P5110)
# ----------------------------------





#### Evaluate accuracy of this model.

In [ ]:
# Add code below this comment  (Question #P5111)
# ----------------------------------







#### [Optional] Make a prediction of any other single image like we did before.

In [ ]:
# Try whatever you want below this comment  (Question #P5112)
# ----------------------------------






# Save your notebook!